In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tasks import NER
import torch

In [5]:
# load phobert-ner đã được fine-tuned
phobert_ner = AutoModelForTokenClassification.from_pretrained("huyenbui117/Covid19_phoNER")
# load tokenizer của phobert
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
labels = NER().get_labels(None)
labels

['O',
 'B-ORGANIZATION',
 'O-ORGANIZATION',
 'I-ORGANIZATION',
 'B-LOCATION',
 'I-LOCATION',
 'B-SYMPTOM_AND_DISEASE',
 'I-SYMPTOM_AND_DISEASE',
 'B-DATE',
 'I-DATE',
 'B-PATIENT_ID',
 'I-PATIENT_ID',
 'B-AGE',
 'I-AGE',
 'B-NAME',
 'I-NAME',
 'B-JOB',
 'I-JOB',
 'B-TRANSPORTATION',
 'I-TRANSPORTATION',
 'B-GENDER']

In [7]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
with open("sentences.txt", 'r', encoding="utf-8") as file:
    for line in file:
        words = rdrsegmenter.tokenize(line)[0]
        sequence=" ".join(words)
        print(sequence)
        tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
        inputs = tokenizer.encode(sequence, return_tensors="pt")

        outputs = phobert_ner(inputs)[0]
        predictions = torch.argmax(outputs, dim=2)

        print([(token, labels[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])
        for i in [(token, labels[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
            print(i)

        print('---------------------------------------------')

Bệnh_nhân số 946401 tên là NHT là phi_công thường_trú tại thôn Thành Trung , xã Phồn_Xương , huyện Yên_Thế , tỉnh Bắc_Giang , thực_hiện test nhanh COVID-19 tại Trung_tâm Y_tế huyện Hữu_Lũng cho kết_quả nghi_ngờ ngày 03/11/2021 .
[('<s>', 'O'), ('Bệnh_nhân', 'O'), ('số', 'O'), ('94@@', 'B-PATIENT_ID'), ('64@@', 'B-PATIENT_ID'), ('01', 'B-PATIENT_ID'), ('tên', 'O'), ('là', 'O'), ('NH@@', 'B-NAME'), ('T', 'B-NAME'), ('la@@', 'O'), ('̀', 'O'), ('phi_công', 'B-JOB'), ('thường_trú', 'O'), ('tại', 'O'), ('thôn', 'B-LOCATION'), ('Thành', 'I-LOCATION'), ('Tru@@', 'I-LOCATION'), ('ng@@', 'O'), (',', 'B-LOCATION'), ('xã', 'I-LOCATION'), ('Ph@@', 'I-LOCATION'), ('ồn_@@', 'I-LOCATION'), ('X@@', 'O'), ('ương@@', 'B-LOCATION'), (',', 'I-LOCATION'), ('huyện', 'O'), ('Yên_@@', 'B-LOCATION'), ('Thế@@', 'I-LOCATION'), (',', 'O'), ('tỉnh', 'O'), ('Bắc_@@', 'O'), ('Giang@@', 'O'), (',', 'O'), ('thực_hiện', 'O'), ('test', 'O'), ('nhanh', 'O'), ('CO@@', 'O'), ('VI@@', 'B-LOCATION'), ('D-@@', 'I-LOCATION'), 